<a href="https://colab.research.google.com/github/Rey1380/WavLM/blob/main/Copy_of_wavLM_base_AGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WavLM_base

In [ ]:
!pip install time

In [ ]:
import time
start = time.time()

In [6]:
#!pip install  transformers 
#!pip install --quiet datasets
!pip install pycm==3.5
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
import torch
import librosa
import IPython.display as ipd
import numpy as np

In [10]:
import os
spk2spch = {}
PATH = "/content/drive/MyDrive/intern/farsdot_enrollment2/"
for fname in next(os.walk(PATH))[2]:
    spk_id, _ = fname.split("F")
    speech_array, sample_rate = librosa.load(PATH + fname)
    STD_n= 0.001
    noise=np.random.normal(0, STD_n, speech_array.shape[0])
    speech_array=noise+speech_array
    speech_array = speech_array.squeeze().astype(np.float)
    speech_array = librosa.resample(np.array(speech_array), sample_rate, 16000)
    
    spk2spch[f"spk-{spk_id}"] = speech_array
print(spk2spch)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


{'spk-019': array([ 0.00018955,  0.00288207,  0.00156846, ..., -0.01427775,
       -0.0177972 , -0.03081423]), 'spk-020': array([ 0.02116822,  0.02917664,  0.01669931, ..., -0.00306366,
       -0.00255543, -0.00438587]), 'spk-001': array([-0.08406139, -0.1254356 , -0.1483495 , ..., -0.00256425,
       -0.00114273, -0.00204607]), 'spk-021': array([0.01639675, 0.01633458, 0.01000406, ..., 0.01261686, 0.01196019,
       0.01579024]), 'spk-022': array([-0.01138564, -0.01373832, -0.0150244 , ..., -0.00884775,
       -0.00879203, -0.00646823]), 'spk-023': array([-0.03847896, -0.0477297 , -0.03763648, ..., -0.02764628,
       -0.03811038, -0.05557559]), 'spk-024': array([0.00253948, 0.00392595, 0.00347412, ..., 0.08400197, 0.09353643,
       0.09191896]), 'spk-026': array([ 0.07149224,  0.0779906 ,  0.04906847, ..., -0.0079127 ,
       -0.02464486, -0.03416422]), 'spk-028': array([-0.08422334, -0.07056539, -0.01144069, ..., -0.00811272,
       -0.01628621,  0.00366303]), 'spk-029': array([ 0.

In [11]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 13.3 MB/s 
     |████████████████████████████████| 141 kB 58.4 MB/s 
     |████████████████████████████████| 212 kB 74.1 MB/s 
     |████████████████████████████████| 115 kB 70.1 MB/s 
     |████████████████████████████████| 127 kB 58.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [13]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
from datasets import load_dataset
import torch
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("microsoft/wavlm-base-plus-sv")
model = WavLMForXVector.from_pretrained("microsoft/wavlm-base-plus-sv")
threshold = 0.9
similarities = []

In [ ]:
from os.path import exists

with open('/content/drive/MyDrive/intern/farsdot/trials') as f:
    for line in f:
        line = line.strip()
        spk_id, utt_id, target_nontarget = line.split()
        path = "/content/drive/MyDrive/intern/wavlm large/farsdot/"  + utt_id.split("-")[1] + ".wav.wav"
        if exists(path) is False:
            path = "/content/drive/MyDrive/intern/wavlm large/farsdot/"  + utt_id.split("-")[1] + ".WAV.WAV"
        
        speech_array2, sample_rate =librosa.load(path)
        print(path, spk_id, target_nontarget)
        speech_array2 = speech_array2.squeeze().astype(float)
        speech_array2 = librosa.resample(np.array(speech_array2), sample_rate, 16000)
        speech3 = [spk2spch[spk_id], speech_array2]
        
        inputs= feature_extractor(
            speech3,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True)

        with torch.no_grad():
            embeddings = model(**inputs).embeddings

        embeddings = torch.nn.functional.normalize(embeddings, dim=-1).cpu()
        cosine_sim = torch.nn.CosineSimilarity(dim=-1)
        similarity = cosine_sim(embeddings[0], embeddings[1])
        similarities.append(similarity.item())
        print(similarity.item())

/content/drive/MyDrive/intern/wavlm large/farsdot/001F02.WAV.WAV spk-001 target


/usr/local/lib/python3.7/dist-packages/transformers/models/wavlm/modeling_wavlm.py:1755: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


0.9759833216667175
/content/drive/MyDrive/intern/wavlm large/farsdot/001F03.WAV.WAV spk-001 target
0.9831200242042542
/content/drive/MyDrive/intern/wavlm large/farsdot/001F04.WAV.WAV spk-001 target
0.9832634925842285
/content/drive/MyDrive/intern/wavlm large/farsdot/001F05.WAV.WAV spk-001 target
0.9568886756896973
/content/drive/MyDrive/intern/wavlm large/farsdot/001F06.WAV.WAV spk-001 target
0.9583774209022522
/content/drive/MyDrive/intern/wavlm large/farsdot/001F07.WAV.WAV spk-001 target
0.9853382110595703
/content/drive/MyDrive/intern/wavlm large/farsdot/001F08.WAV.WAV spk-001 target
0.9743375182151794
/content/drive/MyDrive/intern/wavlm large/farsdot/001F09.WAV.WAV spk-001 target
0.9792191982269287
/content/drive/MyDrive/intern/wavlm large/farsdot/001F10.WAV.WAV spk-001 target
0.9693450927734375
/content/drive/MyDrive/intern/wavlm large/farsdot/002F02.wav.wav spk-001 nontarget
0.7038866281509399
/content/drive/MyDrive/intern/wavlm large/farsdot/002F03.wav.wav spk-001 nontarget
0.76

In [ ]:
np.save("similarities.npy", np.array(similarities))

In [ ]:
actual_vector = []
with open('/content/drive/MyDrive/intern/wavlm large/farsdot/trials') as f:
    for line in f:
        line = line.strip()
        spk_id, utt_id, target_nontarget = line.split()
        if target_nontarget == "nontarget":
            actual_vector.append(0)
        else:
            actual_vector.append(1)
actual_vector = np.array(actual_vector)

ROC:

In [ ]:
threshold = 0.957

In [ ]:
similarities = np.array(similarities)
predict_vector = (similarities > threshold) * 1
print(predict_vector)
print(actual_vector)

In [ ]:
np.save("actual_vector.npy", np.array(actual_vector))

In [ ]:
actual_vector = np.load("actual_vector.npy")
predict_vector = np.load("predict_vector.npy")
similarities = np.load("similarities.npy")

In [ ]:
sum(predict_vector != actual_vector) / len(actual_vector)

In [ ]:
from pycm import ConfusionMatrix
cm = ConfusionMatrix(actual_vector, predict_vector)
cm

In [ ]:
cm.plot()

In [ ]:
print(cm)

In [ ]:
print(cm.overall_stat["F1 Macro"], cm.overall_stat["F1 Micro"])

In [ ]:
for threshold in np.arange(0.94, 0.98, 0.001):
    predict_vector = (similarities > threshold) * 1
    cm = ConfusionMatrix(actual_vector, predict_vector)
    print(threshold, cm.overall_stat["F1 Macro"], cm.overall_stat["F1 Micro"])

In [ ]:
print(speech3)

In [ ]:
inputs= feature_extractor(
  speech3, sampling_rate=16000, return_tensors="pt", padding=True
    )

with torch.no_grad():
    embeddings = model(**inputs).embeddings

embeddings = torch.nn.functional.normalize(embeddings, dim=-1).cpu()

# the resulting embeddings can be used for cosine similarity-based retrieval
cosine_sim = torch.nn.CosineSimilarity(dim=-1)
similarity = cosine_sim(embeddings[0], embeddings[1])
threshold = 0.9  # the optimal threshold is dataset-dependent
if similarity < threshold:
    print("Speakers are not the same!")
round(similarity.item(), 2)

In [ ]:
from pycm import *
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import os

In [ ]:
end = time.time()
print("Execution time of the program is(RTF)- ", end-start)